In [8]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np

In [9]:
test = pd.read_excel('data/Test_unlabled.xlsx')
df = pd.read_excel('data/Train_labled.xlsx')

In [10]:
for i in df.columns:
    print("|{}|, {}, ".format(i, df[i].unique().shape[0]))

|Unnamed: 0.1|, 259, 
|Unnamed: 0|, 259, 
|Возраст|, 49, 
|Пол|, 2, 
|Вес, кг|, 57, 
|Рост, см|, 39, 
|рост2, м2|, 39, 
|BMI, кг/м2|, 212, 
|Болезни, влияющие на ф-цию почек|, 2, 
|Charlson Comorbidity Index, |, 10, 
|Comorbidity Component|, 3, 
|1. Scoring: Age|, 5, 
|ASA (в осмотре анестезиологом)|, 3, 
|Сторона поражения|, 2, 
|d max при одностороннем поражении|, 126, 
|локализация|, 5, 
|локализация.1|, 3, 
|кисты|, 2, 
|RENAL|, 8, 
|RENAL риск (4-6 vs 7-9 vs >=10)|, 3, 
|Размер|, 3, 
|Экзофит >50% - <50% - эндофит|, 3, 
|синус/члс >7, 4-7, <=4 мм|, 3, 
|Polar lines|, 3, 
|PADUA|, 8, 
|PADUA риск (6-7 vs 8-9 vs >=10)|, 3, 
|Longitudinal|, 2, 
|Exophitic|, 3, 
|Renal rim|, 2, 
|Sinus|, 2, 
|ЧЛС|, 2, 
|Размер.1|, 3, 
|Показания к резекции|, 4, 
|Единственная почка не из-за рака|, 2, 
|Нефункционирующая вторая почка не из-за рака|, 2, 
|Нормальная вторая почка|, 2, 
|Аномальная почка|, 3, 
|Исходный диагноз|, 2, 
|Доступ первой резекции|, 2, 
|Доступ первой резекции.1|, 2, 
|Ишемия да

In [11]:
df["Текущий прогноз"].value_counts()

0    167
1     73
2     19
Name: Текущий прогноз, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

(не будем, будем сразу делать мультиклассовую классификацию) Пока будем предсказывать жив или мертв, так что жив с рецедивом (2) заменяем на жив (0) 


Столбцы Unnamed: 0.1	Unnamed: 0	кажется просто индексы строчек, дропаем их


Столбцы M, рN, содержат только одно значение или NaN, дропаем их

заменим все что связано с рецидивом на 0, если рецидив 0

Столбцы "PADUA риск (6-7 vs 8-9 vs >=10)", "RENAL риск (4-6 vs 7-9 vs >=10)" дерево само поймет по соответсв. другим столбцам, дропаем


В столбце эффект лечения рецидива в тесте есть 3, 4, в трейне нет, меняем из на нан, их всего 4 штуки

Категориальными признаками считаем те, которые содержат <4 уникальных значений


В стобцах "местный", "системное лечение", "Гемотрансфузия", "Маннит", "Лазикс" заменим наны на преобладающий класс (0)


В остальных категориальных признаках ("мтс", "эффект лечения рецидива") заменим наны на отдельное значение -1, попробуем вытянуть из этого полезную информацию

In [13]:
#df.loc[df["Текущий прогноз"] == 2, "Текущий прогноз"] = 0
to_drop = ["Unnamed: 0.1", "Unnamed: 0", "M", "рN"]
to_drop_tree = ["PADUA риск (6-7 vs 8-9 vs >=10)", "RENAL риск (4-6 vs 7-9 vs >=10)"]

to_drop = to_drop_tree + to_drop
df.drop(axis=1, labels=to_drop, inplace=True, errors="ignore")
test.drop(axis=1, labels=to_drop, inplace=True, errors="ignore")
# заменим все что связано с рецидивом на 0, если рецидив 0
recediv_col = ["вид рецидива", "Лечение рецидива", "эффект лечения рецидива"]
df.loc[df["Рецидив"] == 0, recediv_col] = 0
test.loc[test["Рецидив"] == 0, recediv_col] = 0

test.loc[test["эффект лечения рецидива"].isin([3, 4]), "эффект лечения рецидива"] = np.nan

to_fill_zero = ["местный", "системное лечение", "Гемотрансфузия", "Маннит", "Лазикс"]
test[to_fill_zero] = test[to_fill_zero].fillna(0)
df[to_fill_zero] = df[to_fill_zero].fillna(0)

to_fill_one = ["pT при одностороннем поражении.1"]
test[to_fill_one] = test[to_fill_one].fillna(0)
df[to_fill_one] = df[to_fill_one].fillna(0)

new_category = ["мтс", "эффект лечения рецидива", "вид рецидива", "Лечение рецидива"]

test[new_category] = test[new_category].fillna(-1)
df[new_category] = df[new_category].fillna(-1)


In [14]:
y = "Текущий прогноз"
model_feat = [col for col in df.columns if (col != y)]
category_feat = [col for col in model_feat if df[col].unique().shape[0] <= 4] 

In [15]:
print(*sorted(list(zip(df.columns, df.corrwith(df[y]))),key=lambda x : -x[1]), sep="\n")

('Время операции', 0.5800311963425582)
('Швы через толщу паренхимы', 0.37604691430846404)
('Срочная гистология', 0.344555628687669)
('Sinus', 0.24985517969894339)
('Единственная почка не из-за рака', 0.2420424894365287)
('Отступ от края опухоли, мм', 0.22724177024423586)
('рост2, м2', 0.22072868743298496)
('Рост, см', 0.21976981553354394)
('ASA (в осмотре анестезиологом)', 0.21331309031321335)
('Что пережимали', 0.15092863786564167)
('Размер', 0.13756001763860637)
('d max при одностороннем поражении', 0.12587636832000337)
('Renal rim', 0.1233817427810554)
('ЧЛС', 0.12260710708828854)
('Размер.1', 0.11229824885948766)
('Пол', 0.10605347781211225)
('PADUA', 0.09455593079638228)
('RENAL', 0.09259379657428754)
('Exophitic', 0.0912094108323644)
('синус/члс >7, 4-7, <=4 мм', 0.08497293739238765)
('%удаленной паренхимы', 0.07935506612122575)
('Экзофит >50% - <50% - эндофит', 0.07113827813235772)
('Возраст', 0.07045802422716219)
('Время ишемии', 0.025214878361955274)
('1. Scoring: Age', 0.0194

In [16]:
for i in category_feat:
    if df[i].isnull().sum() != 0:
        print("{}, {}".format(i, df[i].value_counts(normalize=True, dropna=False)))

In [17]:
for col in category_feat:
    df[col] = df[col].astype(str)
    test[col] = test[col].astype(str)

In [18]:
test.shape, df.shape

((112, 111), (259, 112))

In [19]:
train, val = train_test_split(df, test_size=0.6, random_state=42)

In [20]:
model = CatBoostClassifier(auto_class_weights="Balanced", boosting_type="Ordered", learning_rate=0.1)
model.fit(train[model_feat], y=train[y].values, eval_set=(val[model_feat], val[y].values), 
                 early_stopping_rounds=20, 
                 cat_features=category_feat)

0:	learn: 0.9998328	test: 1.0066313	best: 1.0066313 (0)	total: 254ms	remaining: 4m 13s
1:	learn: 0.8930635	test: 0.8926849	best: 0.8926849 (1)	total: 335ms	remaining: 2m 46s
2:	learn: 0.8283816	test: 0.8522894	best: 0.8522894 (2)	total: 526ms	remaining: 2m 54s
3:	learn: 0.7913703	test: 0.8167138	best: 0.8167138 (3)	total: 763ms	remaining: 3m 9s
4:	learn: 0.7183448	test: 0.7424696	best: 0.7424696 (4)	total: 905ms	remaining: 3m
5:	learn: 0.6538906	test: 0.6734792	best: 0.6734792 (5)	total: 1s	remaining: 2m 46s
6:	learn: 0.6123226	test: 0.6299601	best: 0.6299601 (6)	total: 1.22s	remaining: 2m 52s
7:	learn: 0.5701786	test: 0.5956209	best: 0.5956209 (7)	total: 1.35s	remaining: 2m 47s
8:	learn: 0.5426802	test: 0.5648701	best: 0.5648701 (8)	total: 1.56s	remaining: 2m 52s
9:	learn: 0.5110617	test: 0.5327787	best: 0.5327787 (9)	total: 1.61s	remaining: 2m 39s
10:	learn: 0.4743343	test: 0.4950928	best: 0.4950928 (10)	total: 1.83s	remaining: 2m 44s
11:	learn: 0.4533218	test: 0.4705661	best: 0.4705

93:	learn: 0.0506038	test: 0.0543736	best: 0.0543736 (93)	total: 15.1s	remaining: 2m 25s
94:	learn: 0.0499635	test: 0.0539089	best: 0.0539089 (94)	total: 15.3s	remaining: 2m 26s
95:	learn: 0.0493802	test: 0.0533416	best: 0.0533416 (95)	total: 15.6s	remaining: 2m 27s
96:	learn: 0.0488629	test: 0.0530216	best: 0.0530216 (96)	total: 15.8s	remaining: 2m 27s
97:	learn: 0.0482459	test: 0.0521647	best: 0.0521647 (97)	total: 16.1s	remaining: 2m 28s
98:	learn: 0.0477660	test: 0.0519054	best: 0.0519054 (98)	total: 16.4s	remaining: 2m 28s
99:	learn: 0.0473138	test: 0.0517861	best: 0.0517861 (99)	total: 16.6s	remaining: 2m 29s
100:	learn: 0.0468782	test: 0.0515613	best: 0.0515613 (100)	total: 16.9s	remaining: 2m 30s
101:	learn: 0.0462156	test: 0.0512732	best: 0.0512732 (101)	total: 17.1s	remaining: 2m 30s
102:	learn: 0.0457837	test: 0.0509956	best: 0.0509956 (102)	total: 17.4s	remaining: 2m 31s
103:	learn: 0.0452090	test: 0.0506221	best: 0.0506221 (103)	total: 17.6s	remaining: 2m 31s
104:	learn: 0

184:	learn: 0.0233151	test: 0.0339241	best: 0.0339241 (184)	total: 35.9s	remaining: 2m 38s
185:	learn: 0.0231944	test: 0.0338463	best: 0.0338463 (185)	total: 36.1s	remaining: 2m 37s
186:	learn: 0.0230415	test: 0.0337014	best: 0.0337014 (186)	total: 36.3s	remaining: 2m 37s
187:	learn: 0.0229280	test: 0.0336125	best: 0.0336125 (187)	total: 36.6s	remaining: 2m 37s
188:	learn: 0.0228246	test: 0.0334993	best: 0.0334993 (188)	total: 36.8s	remaining: 2m 37s
189:	learn: 0.0226784	test: 0.0333174	best: 0.0333174 (189)	total: 37s	remaining: 2m 37s
190:	learn: 0.0225468	test: 0.0332076	best: 0.0332076 (190)	total: 37.2s	remaining: 2m 37s
191:	learn: 0.0223650	test: 0.0330620	best: 0.0330620 (191)	total: 37.5s	remaining: 2m 37s
192:	learn: 0.0222338	test: 0.0329079	best: 0.0329079 (192)	total: 37.7s	remaining: 2m 37s
193:	learn: 0.0220763	test: 0.0327096	best: 0.0327096 (193)	total: 37.9s	remaining: 2m 37s
194:	learn: 0.0219531	test: 0.0325585	best: 0.0325585 (194)	total: 38.2s	remaining: 2m 37s
1

275:	learn: 0.0142619	test: 0.0255927	best: 0.0255927 (275)	total: 57.7s	remaining: 2m 31s
276:	learn: 0.0141774	test: 0.0255206	best: 0.0255206 (276)	total: 57.9s	remaining: 2m 31s
277:	learn: 0.0141101	test: 0.0254647	best: 0.0254647 (277)	total: 58.2s	remaining: 2m 31s
278:	learn: 0.0140447	test: 0.0254238	best: 0.0254238 (278)	total: 58.5s	remaining: 2m 31s
279:	learn: 0.0139884	test: 0.0253748	best: 0.0253748 (279)	total: 58.7s	remaining: 2m 30s
280:	learn: 0.0139185	test: 0.0253393	best: 0.0253393 (280)	total: 59s	remaining: 2m 30s
281:	learn: 0.0138437	test: 0.0252387	best: 0.0252387 (281)	total: 59.2s	remaining: 2m 30s
282:	learn: 0.0137996	test: 0.0251714	best: 0.0251714 (282)	total: 59.5s	remaining: 2m 30s
283:	learn: 0.0137491	test: 0.0251308	best: 0.0251308 (283)	total: 59.7s	remaining: 2m 30s
284:	learn: 0.0136888	test: 0.0251014	best: 0.0251014 (284)	total: 1m	remaining: 2m 30s
285:	learn: 0.0136366	test: 0.0250510	best: 0.0250510 (285)	total: 1m	remaining: 2m 30s
286:	le

365:	learn: 0.0102592	test: 0.0214981	best: 0.0214981 (365)	total: 1m 18s	remaining: 2m 16s
366:	learn: 0.0102317	test: 0.0214688	best: 0.0214688 (366)	total: 1m 18s	remaining: 2m 16s
367:	learn: 0.0101863	test: 0.0214026	best: 0.0214026 (367)	total: 1m 19s	remaining: 2m 15s
368:	learn: 0.0101601	test: 0.0213816	best: 0.0213816 (368)	total: 1m 19s	remaining: 2m 15s
369:	learn: 0.0101297	test: 0.0213581	best: 0.0213581 (369)	total: 1m 19s	remaining: 2m 15s
370:	learn: 0.0100996	test: 0.0213317	best: 0.0213317 (370)	total: 1m 19s	remaining: 2m 15s
371:	learn: 0.0100764	test: 0.0213085	best: 0.0213085 (371)	total: 1m 20s	remaining: 2m 15s
372:	learn: 0.0100384	test: 0.0212749	best: 0.0212749 (372)	total: 1m 20s	remaining: 2m 14s
373:	learn: 0.0100010	test: 0.0212536	best: 0.0212536 (373)	total: 1m 20s	remaining: 2m 14s
374:	learn: 0.0099637	test: 0.0212427	best: 0.0212427 (374)	total: 1m 20s	remaining: 2m 14s
375:	learn: 0.0099291	test: 0.0212106	best: 0.0212106 (375)	total: 1m 20s	remain

...

546:	learn: 0.0064150	test: 0.0173239	best: 0.0173239 (546)	total: 1m 58s	remaining: 1m 38s
547:	learn: 0.0063981	test: 0.0173078	best: 0.0173078 (547)	total: 1m 59s	remaining: 1m 38s
548:	learn: 0.0063796	test: 0.0172807	best: 0.0172807 (548)	total: 1m 59s	remaining: 1m 37s
549:	learn: 0.0063648	test: 0.0172488	best: 0.0172488 (549)	total: 1m 59s	remaining: 1m 37s
550:	learn: 0.0063516	test: 0.0172400	best: 0.0172400 (550)	total: 1m 59s	remaining: 1m 37s
551:	learn: 0.0063386	test: 0.0172301	best: 0.0172301 (551)	total: 1m 59s	remaining: 1m 37s
552:	learn: 0.0063227	test: 0.0172131	best: 0.0172131 (552)	total: 2m	remaining: 1m 37s
553:	learn: 0.0063109	test: 0.0171762	best: 0.0171762 (553)	total: 2m	remaining: 1m 36s
554:	learn: 0.0063048	test: 0.0171772	best: 0.0171762 (553)	total: 2m	remaining: 1m 36s
555:	learn: 0.0062941	test: 0.0171651	best: 0.0171651 (555)	total: 2m	remaining: 1m 36s
556:	learn: 0.0062870	test: 0.0171536	best: 0.0171536 (556)	total: 2m	remaining: 1m 36s
557:	lea

636:	learn: 0.0053985	test: 0.0161015	best: 0.0161015 (636)	total: 2m 18s	remaining: 1m 19s
637:	learn: 0.0053886	test: 0.0160894	best: 0.0160894 (637)	total: 2m 19s	remaining: 1m 18s
638:	learn: 0.0053783	test: 0.0160789	best: 0.0160789 (638)	total: 2m 19s	remaining: 1m 18s
639:	learn: 0.0053669	test: 0.0160599	best: 0.0160599 (639)	total: 2m 19s	remaining: 1m 18s
640:	learn: 0.0053574	test: 0.0160570	best: 0.0160570 (640)	total: 2m 19s	remaining: 1m 18s
641:	learn: 0.0053516	test: 0.0160529	best: 0.0160529 (641)	total: 2m 20s	remaining: 1m 18s
642:	learn: 0.0053417	test: 0.0160430	best: 0.0160430 (642)	total: 2m 20s	remaining: 1m 17s
643:	learn: 0.0053317	test: 0.0160365	best: 0.0160365 (643)	total: 2m 20s	remaining: 1m 17s
644:	learn: 0.0053249	test: 0.0160356	best: 0.0160356 (644)	total: 2m 20s	remaining: 1m 17s
645:	learn: 0.0053173	test: 0.0160293	best: 0.0160293 (645)	total: 2m 21s	remaining: 1m 17s
646:	learn: 0.0053111	test: 0.0160181	best: 0.0160181 (646)	total: 2m 21s	remain

726:	learn: 0.0046693	test: 0.0152743	best: 0.0152743 (726)	total: 2m 38s	remaining: 59.5s
727:	learn: 0.0046609	test: 0.0152687	best: 0.0152687 (727)	total: 2m 38s	remaining: 59.3s
728:	learn: 0.0046558	test: 0.0152623	best: 0.0152623 (728)	total: 2m 38s	remaining: 59.1s
729:	learn: 0.0046497	test: 0.0152421	best: 0.0152421 (729)	total: 2m 39s	remaining: 58.9s
730:	learn: 0.0046430	test: 0.0152412	best: 0.0152412 (730)	total: 2m 39s	remaining: 58.6s
731:	learn: 0.0046370	test: 0.0152262	best: 0.0152262 (731)	total: 2m 39s	remaining: 58.4s
732:	learn: 0.0046311	test: 0.0152181	best: 0.0152181 (732)	total: 2m 39s	remaining: 58.2s
733:	learn: 0.0046244	test: 0.0152114	best: 0.0152114 (733)	total: 2m 40s	remaining: 58s
734:	learn: 0.0046176	test: 0.0152071	best: 0.0152071 (734)	total: 2m 40s	remaining: 57.8s
735:	learn: 0.0046102	test: 0.0151957	best: 0.0151957 (735)	total: 2m 40s	remaining: 57.6s
736:	learn: 0.0046021	test: 0.0151847	best: 0.0151847 (736)	total: 2m 40s	remaining: 57.4s
7

818:	learn: 0.0040942	test: 0.0144387	best: 0.0144387 (818)	total: 2m 58s	remaining: 39.5s
819:	learn: 0.0040870	test: 0.0144396	best: 0.0144387 (818)	total: 2m 59s	remaining: 39.3s
820:	learn: 0.0040793	test: 0.0144346	best: 0.0144346 (820)	total: 2m 59s	remaining: 39.1s
821:	learn: 0.0040733	test: 0.0144174	best: 0.0144174 (821)	total: 2m 59s	remaining: 38.9s
822:	learn: 0.0040663	test: 0.0144110	best: 0.0144110 (822)	total: 2m 59s	remaining: 38.7s
823:	learn: 0.0040602	test: 0.0144163	best: 0.0144110 (822)	total: 2m 59s	remaining: 38.4s
824:	learn: 0.0040542	test: 0.0144102	best: 0.0144102 (824)	total: 3m	remaining: 38.2s
825:	learn: 0.0040499	test: 0.0144058	best: 0.0144058 (825)	total: 3m	remaining: 38s
826:	learn: 0.0040451	test: 0.0143994	best: 0.0143994 (826)	total: 3m	remaining: 37.8s
827:	learn: 0.0040416	test: 0.0143997	best: 0.0143994 (826)	total: 3m	remaining: 37.6s
828:	learn: 0.0040370	test: 0.0143922	best: 0.0143922 (828)	total: 3m 1s	remaining: 37.4s
829:	learn: 0.0040

910:	learn: 0.0036235	test: 0.0138976	best: 0.0138976 (910)	total: 3m 19s	remaining: 19.5s
911:	learn: 0.0036193	test: 0.0139000	best: 0.0138976 (910)	total: 3m 19s	remaining: 19.2s
912:	learn: 0.0036137	test: 0.0138800	best: 0.0138800 (912)	total: 3m 19s	remaining: 19s
913:	learn: 0.0036075	test: 0.0138768	best: 0.0138768 (913)	total: 3m 19s	remaining: 18.8s
914:	learn: 0.0036024	test: 0.0138712	best: 0.0138712 (914)	total: 3m 20s	remaining: 18.6s
915:	learn: 0.0035993	test: 0.0138723	best: 0.0138712 (914)	total: 3m 20s	remaining: 18.4s
916:	learn: 0.0035916	test: 0.0138622	best: 0.0138622 (916)	total: 3m 20s	remaining: 18.1s
917:	learn: 0.0035866	test: 0.0138514	best: 0.0138514 (917)	total: 3m 20s	remaining: 17.9s
918:	learn: 0.0035837	test: 0.0138413	best: 0.0138413 (918)	total: 3m 20s	remaining: 17.7s
919:	learn: 0.0035775	test: 0.0138284	best: 0.0138284 (919)	total: 3m 21s	remaining: 17.5s
920:	learn: 0.0035743	test: 0.0138269	best: 0.0138269 (920)	total: 3m 21s	remaining: 17.3s
9

посчитаем качество полученной модели:

In [21]:
from sklearn.metrics import balanced_accuracy_score
val_y = model.predict(val[model_feat])
balanced_accuracy_score(val_y, val[y])

0.9743589743589745

In [345]:
val[y].value_counts(normalize=True), train[y].value_counts(normalize=True)

(0    0.660256
 1    0.262821
 2    0.076923
 Name: Текущий прогноз, dtype: float64,
 0    0.621359
 1    0.310680
 2    0.067961
 Name: Текущий прогноз, dtype: float64)

обучим на всем датасете и посмотрим важность фичей:

In [22]:
model = CatBoostClassifier(iterations= 999, auto_class_weights="Balanced", boosting_type="Ordered", learning_rate=0.1)
model.fit(df[model_feat], y=df[y].values, cat_features=category_feat)

0:	learn: 0.9941704	total: 84.4ms	remaining: 1m 24s
1:	learn: 0.8904105	total: 354ms	remaining: 2m 56s
2:	learn: 0.7877695	total: 455ms	remaining: 2m 31s
3:	learn: 0.7067999	total: 774ms	remaining: 3m 12s
4:	learn: 0.6352353	total: 875ms	remaining: 2m 54s
5:	learn: 0.5728653	total: 1.05s	remaining: 2m 53s
6:	learn: 0.5172603	total: 1.11s	remaining: 2m 38s
7:	learn: 0.4704907	total: 1.42s	remaining: 2m 55s
8:	learn: 0.4346749	total: 1.73s	remaining: 3m 9s
9:	learn: 0.4030133	total: 2.09s	remaining: 3m 26s
10:	learn: 0.3736932	total: 2.42s	remaining: 3m 37s
11:	learn: 0.3454595	total: 2.58s	remaining: 3m 32s
12:	learn: 0.3188838	total: 2.88s	remaining: 3m 38s
13:	learn: 0.2983552	total: 3.22s	remaining: 3m 46s
14:	learn: 0.2779073	total: 3.56s	remaining: 3m 53s
15:	learn: 0.2588728	total: 3.66s	remaining: 3m 45s
16:	learn: 0.2399089	total: 3.85s	remaining: 3m 42s
17:	learn: 0.2251593	total: 3.97s	remaining: 3m 36s
18:	learn: 0.2106806	total: 4.04s	remaining: 3m 28s
19:	learn: 0.1977085	t

158:	learn: 0.0158038	total: 43s	remaining: 3m 47s
159:	learn: 0.0157004	total: 43.4s	remaining: 3m 47s
160:	learn: 0.0155685	total: 43.7s	remaining: 3m 47s
161:	learn: 0.0154573	total: 44s	remaining: 3m 47s
162:	learn: 0.0153709	total: 44.3s	remaining: 3m 47s
163:	learn: 0.0152737	total: 44.7s	remaining: 3m 47s
164:	learn: 0.0152037	total: 45s	remaining: 3m 47s
165:	learn: 0.0149979	total: 45.3s	remaining: 3m 47s
166:	learn: 0.0148562	total: 45.6s	remaining: 3m 47s
167:	learn: 0.0147845	total: 46s	remaining: 3m 47s
168:	learn: 0.0146613	total: 46.3s	remaining: 3m 47s
169:	learn: 0.0144704	total: 46.6s	remaining: 3m 47s
170:	learn: 0.0143706	total: 47s	remaining: 3m 47s
171:	learn: 0.0141476	total: 47.3s	remaining: 3m 47s
172:	learn: 0.0140816	total: 47.7s	remaining: 3m 47s
173:	learn: 0.0139659	total: 48.1s	remaining: 3m 47s
174:	learn: 0.0137748	total: 48.5s	remaining: 3m 48s
175:	learn: 0.0136109	total: 48.9s	remaining: 3m 48s
176:	learn: 0.0135092	total: 49.3s	remaining: 3m 49s
177

312:	learn: 0.0060544	total: 1m 42s	remaining: 3m 43s
313:	learn: 0.0060203	total: 1m 42s	remaining: 3m 43s
314:	learn: 0.0060004	total: 1m 42s	remaining: 3m 43s
315:	learn: 0.0059720	total: 1m 43s	remaining: 3m 43s
316:	learn: 0.0059439	total: 1m 43s	remaining: 3m 43s
317:	learn: 0.0059300	total: 1m 44s	remaining: 3m 43s
318:	learn: 0.0059018	total: 1m 44s	remaining: 3m 43s
319:	learn: 0.0058741	total: 1m 44s	remaining: 3m 42s
320:	learn: 0.0058397	total: 1m 45s	remaining: 3m 42s
321:	learn: 0.0058179	total: 1m 45s	remaining: 3m 42s
322:	learn: 0.0058049	total: 1m 46s	remaining: 3m 41s
323:	learn: 0.0057743	total: 1m 46s	remaining: 3m 41s
324:	learn: 0.0057463	total: 1m 46s	remaining: 3m 41s
325:	learn: 0.0057255	total: 1m 47s	remaining: 3m 41s
326:	learn: 0.0057126	total: 1m 47s	remaining: 3m 40s
327:	learn: 0.0056971	total: 1m 47s	remaining: 3m 40s
328:	learn: 0.0056775	total: 1m 48s	remaining: 3m 40s
329:	learn: 0.0056652	total: 1m 48s	remaining: 3m 40s
330:	learn: 0.0056391	total:

465:	learn: 0.0037184	total: 2m 40s	remaining: 3m 3s
466:	learn: 0.0037131	total: 2m 41s	remaining: 3m 3s
467:	learn: 0.0037021	total: 2m 41s	remaining: 3m 3s
468:	learn: 0.0036912	total: 2m 41s	remaining: 3m 2s
469:	learn: 0.0036854	total: 2m 42s	remaining: 3m 2s
470:	learn: 0.0036767	total: 2m 42s	remaining: 3m 2s
471:	learn: 0.0036683	total: 2m 43s	remaining: 3m 2s
472:	learn: 0.0036580	total: 2m 43s	remaining: 3m 1s
473:	learn: 0.0036464	total: 2m 43s	remaining: 3m 1s
474:	learn: 0.0036371	total: 2m 44s	remaining: 3m 1s
475:	learn: 0.0036254	total: 2m 44s	remaining: 3m
476:	learn: 0.0036145	total: 2m 45s	remaining: 3m
477:	learn: 0.0036020	total: 2m 45s	remaining: 3m
478:	learn: 0.0035937	total: 2m 45s	remaining: 3m
479:	learn: 0.0035846	total: 2m 46s	remaining: 2m 59s
480:	learn: 0.0035718	total: 2m 46s	remaining: 2m 59s
481:	learn: 0.0035627	total: 2m 47s	remaining: 2m 59s
482:	learn: 0.0035513	total: 2m 47s	remaining: 2m 59s
483:	learn: 0.0035422	total: 2m 48s	remaining: 2m 58s


618:	learn: 0.0026785	total: 3m 40s	remaining: 2m 15s
619:	learn: 0.0026753	total: 3m 41s	remaining: 2m 15s
620:	learn: 0.0026701	total: 3m 41s	remaining: 2m 14s
621:	learn: 0.0026656	total: 3m 41s	remaining: 2m 14s
622:	learn: 0.0026589	total: 3m 42s	remaining: 2m 14s
623:	learn: 0.0026523	total: 3m 42s	remaining: 2m 13s
624:	learn: 0.0026468	total: 3m 42s	remaining: 2m 13s
625:	learn: 0.0026435	total: 3m 43s	remaining: 2m 13s
626:	learn: 0.0026388	total: 3m 43s	remaining: 2m 12s
627:	learn: 0.0026344	total: 3m 44s	remaining: 2m 12s
628:	learn: 0.0026289	total: 3m 44s	remaining: 2m 12s
629:	learn: 0.0026253	total: 3m 44s	remaining: 2m 11s
630:	learn: 0.0026214	total: 3m 45s	remaining: 2m 11s
631:	learn: 0.0026156	total: 3m 45s	remaining: 2m 10s
632:	learn: 0.0026078	total: 3m 45s	remaining: 2m 10s
633:	learn: 0.0026032	total: 3m 46s	remaining: 2m 10s
634:	learn: 0.0025989	total: 3m 46s	remaining: 2m 9s
635:	learn: 0.0025941	total: 3m 46s	remaining: 2m 9s
636:	learn: 0.0025864	total: 3

772:	learn: 0.0020754	total: 4m 40s	remaining: 1m 21s
773:	learn: 0.0020719	total: 4m 40s	remaining: 1m 21s
774:	learn: 0.0020645	total: 4m 40s	remaining: 1m 21s
775:	learn: 0.0020612	total: 4m 41s	remaining: 1m 20s
776:	learn: 0.0020577	total: 4m 41s	remaining: 1m 20s
777:	learn: 0.0020550	total: 4m 42s	remaining: 1m 20s
778:	learn: 0.0020524	total: 4m 42s	remaining: 1m 19s
779:	learn: 0.0020480	total: 4m 42s	remaining: 1m 19s
780:	learn: 0.0020450	total: 4m 43s	remaining: 1m 19s
781:	learn: 0.0020422	total: 4m 43s	remaining: 1m 18s
782:	learn: 0.0020385	total: 4m 44s	remaining: 1m 18s
783:	learn: 0.0020348	total: 4m 44s	remaining: 1m 18s
784:	learn: 0.0020313	total: 4m 44s	remaining: 1m 17s
785:	learn: 0.0020282	total: 4m 45s	remaining: 1m 17s
786:	learn: 0.0020260	total: 4m 45s	remaining: 1m 16s
787:	learn: 0.0020246	total: 4m 46s	remaining: 1m 16s
788:	learn: 0.0020219	total: 4m 46s	remaining: 1m 16s
789:	learn: 0.0020202	total: 4m 46s	remaining: 1m 15s
790:	learn: 0.0020177	total:

928:	learn: 0.0016884	total: 5m 44s	remaining: 25.9s
929:	learn: 0.0016861	total: 5m 44s	remaining: 25.6s
930:	learn: 0.0016846	total: 5m 45s	remaining: 25.2s
931:	learn: 0.0016831	total: 5m 45s	remaining: 24.8s
932:	learn: 0.0016814	total: 5m 45s	remaining: 24.5s
933:	learn: 0.0016803	total: 5m 46s	remaining: 24.1s
934:	learn: 0.0016791	total: 5m 46s	remaining: 23.7s
935:	learn: 0.0016774	total: 5m 47s	remaining: 23.4s
936:	learn: 0.0016759	total: 5m 47s	remaining: 23s
937:	learn: 0.0016726	total: 5m 47s	remaining: 22.6s
938:	learn: 0.0016698	total: 5m 48s	remaining: 22.3s
939:	learn: 0.0016681	total: 5m 48s	remaining: 21.9s
940:	learn: 0.0016646	total: 5m 48s	remaining: 21.5s
941:	learn: 0.0016631	total: 5m 49s	remaining: 21.1s
942:	learn: 0.0016607	total: 5m 49s	remaining: 20.8s
943:	learn: 0.0016593	total: 5m 50s	remaining: 20.4s
944:	learn: 0.0016578	total: 5m 50s	remaining: 20s
945:	learn: 0.0016558	total: 5m 50s	remaining: 19.7s
946:	learn: 0.0016539	total: 5m 51s	remaining: 19.

In [23]:
print(sorted(list(zip(model_feat, model.get_feature_importance())), key=lambda x : -x[1]))

[('мтс', 38.6299027392719), ('диурез, сутки', 8.30115450733311), ('Время ишемии', 5.439404164252262), ('хирургические ранние структура', 4.578066314650453), ('системное лечение', 4.159440310558754), ('гистологический отступ мм', 3.6138778872452044), ('локализация', 3.387099130287496), ('Время операции', 2.7817370617431494), ('креатинин max', 2.2197967861108), ('PADUA', 2.1458157507061313), ('Кровопотеря', 2.0315832914933405), ('RENAL', 2.0155063080380438), ('Рецидив', 1.938526175873909), ('эффект лечения рецидива', 1.8322201613016702), ('pT при одностороннем поражении', 1.5850029121592994), ('t от диагноза до лечения, мес', 1.3558721342556697), ('локализация.1', 1.157527052019989), ('ОЦК', 0.8659239803224316), ('Лечение рецидива', 0.8480530660882809), ('Размер', 0.7504094269270497), ('Что пережимали', 0.6969764302331721), ('темп диуреза, мл/кг/ч', 0.576549961832156), ('Исходный v почки', 0.5491475197519934), ('Осложнения интраоперационные.1', 0.4846350643609057), ('K+ max', 0.483083610

In [27]:
test['Текущий прогноз'] = model.predict(test[model_feat])

In [28]:
test['Текущий прогноз'].value_counts()

0    73
1    31
2     8
Name: Текущий прогноз, dtype: int64

In [29]:
test.to_csv("live_dead_simple_catboost.csv", index=None)